In [1]:

import os
import importlib
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, Trainer, TrainingArguments, TrainerCallback, EarlyStoppingCallback
import torch
from peft import LoraConfig, get_peft_model
from datasets import Dataset
import json

import prompts
import evaluate

MODEL = 'llama'
DATA = 'acl_arc' #acl_arc, finecite, multicite
SCHEMA = 'XML' # XML, JSON
DATA_SIZE = None # 1000


INPUT = f'./data/{DATA}/{SCHEMA}/'
OUTPUT = f'./output/{DATA}/{MODEL}/{SCHEMA}/'
if DATA_SIZE:
    INPUT = f'./data/{DATA}/{DATA_SIZE}/{SCHEMA}/'
    OUTPUT = f'./output/{DATA}/{MODEL}/{DATA_SIZE}/{SCHEMA}/'

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_mapping = {
    "mistral":'mistralai/Mistral-7B-Instruct-v0.3',
    "scitulu":'allenai/scitulu-7b',
    'llama':'meta-llama/Meta-Llama-3.1-8B-Instruct',
}
model_id = model_mapping[MODEL]

max_seq_length = 1024

#tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token_id = tokenizer.eos_token_id

prompt = prompts.PromptForAutoCCA(tokenizer, MODEL, DATA, SCHEMA)

print(tokenizer.decode(prompt.create_sample('', '',)['input_ids']))

/home/explorer/anaconda3/envs/lasse/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

You are given a list of citation function classes and an excerpt from a scientific text in which citations are marked #REF and #TARGET_REF. Assign the correct function classes to the #TARGET_REF. To do this, first, locate #TARGET_REF, second, select the best fitting function class by considering the target citation context, third, reflect on whether you made the correct choice and respond when you are sure it is correct. Reply by adding a self closing XML tag containing the right function class directly after the #TARGET_REF (e.g: #TARGET_REF<USE/>) 

List Citation Function Classes

                BACK: Citation provides background Information. 

                USE: Citation is used in citing paper. 

                COMP: Cited paper is judged as similar, different, or is agreed or disagreed with. 

          